In [ ]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import util
import datasets2

In [ ]:
# settings
rep = 20

functions = {
             'portraits': datasets2.load_Portraits,
             'mnist': datasets2.load_RotatedMNIST2,
             'cover': datasets2.load_CoverType,
             }
             #'gas': datasets2.load_GasSensor,}

In [ ]:
for name in functions.keys():
    # laod dataset
    load_f = functions[name]
    x_all, y_all = load_f() if name != 'cover' else load_f(sampling=5000)
    x_eval, y_eval = x_all.pop(), y_all.pop()
    # initialize
    num_inter = np.arange(1, len(x_all)-1)
    wd_mean = np.full(shape=(len(num_inter), rep), fill_value=np.nan)
    # calc wd
    print(name)
    for n in tqdm(num_inter):
        for r in range(rep):
            np.random.seed(r)
            torch.manual_seed(r)
            x_subset, y_subset = util.subset_domain(x_all, y_all, n, r)
            mpc_wd = util.mpcWD(x_subset, y_subset)
            wd_mean[n-1, r] = np.mean(mpc_wd)
    # save result
    obj = {'num_inter': num_inter, 'wd': wd_mean}
    pd.to_pickle(obj, f'./result/wd_{name}.pkl')